<a href="https://colab.research.google.com/github/sightes/COFFETALKS/blob/main/afterclass_24_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://raw.githubusercontent.com/sightes/COFFETALKS/main/CLASE%2028%20%20data%20wrangling%20II/p_data.csv'
!wget 'https://raw.githubusercontent.com/sightes/COFFETALKS/main/CLASE%2028%20%20data%20wrangling%20II/w_data.csv'

--2022-02-24 23:03:41--  https://raw.githubusercontent.com/sightes/COFFETALKS/main/CLASE%2028%20%20data%20wrangling%20II/p_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4105795 (3.9M) [text/plain]
Saving to: ‘p_data.csv’

p_data.csv          100%[===================>]   3.92M  --.-KB/s    in 0.09s   

2022-02-24 23:03:42 (44.2 MB/s) - ‘p_data.csv’ saved [4105795/4105795]

--2022-02-24 23:03:42--  https://raw.githubusercontent.com/sightes/COFFETALKS/main/CLASE%2028%20%20data%20wrangling%20II/w_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiti

In [2]:
import pandas as pd 
p=pd.read_csv('p_data.csv', sep=';')
w=pd.read_csv('w_data.csv', sep=';')

In [5]:
print('variables de p:')
print(p.columns)
print('variables de w:')
print(w.columns) 

variables de p:
Index(['Age', 'Sex', 'ID', 'Race', 'Country', 'Education', 'Education-Num',
       'Marital Status', 'Relationship'],
      dtype='object')
variables de w:
Index(['Workclass', 'fnlwgt', 'Capital Gain', 'Capital Loss', 'Hours per week',
       'ID', 'Target'],
      dtype='object')


In [12]:
print(p.nunique())

print('total filas:' + str(len(p)))

Age                  74
Sex                   2
ID                48842
Race                  5
Country              42
Education            16
Education-Num        16
Marital Status        7
Relationship          6
dtype: int64
total filas:48842


In [13]:
print(w.nunique())

print('total filas:' + str(len(w)))

Workclass             9
fnlwgt            28523
Capital Gain        123
Capital Loss         99
Hours per week       96
ID                48843
Target                4
dtype: int64
total filas:48843


In [19]:
print('nulos variable ID en p : ' + str(p.ID.isnull().sum()))
print('nulos variable ID en w : ' + str(w.ID.isnull().sum()))

nulos en p : 0
nulos en w : 0


In [30]:
#opcion con numpy
import numpy as np
#busca los valores en comun p->w
np.where(np.isin(p.ID, w.ID ))

#busca lo que no tienen en comun p->w
np.where((np.isin(p.ID, w.ID )*1)==0)

#busca los valores en comun p->w
np.where(np.isin(w.ID, p.ID ))

#busca lo que no tienen en comun p->w
np.where((np.isin(w.ID, p.ID )*1)==0)

(array([48842]),)

In [31]:
w.iloc[48842]

Workclass          Self-emp-inc
fnlwgt                 182148.0
Capital Gain                0.0
Capital Loss                0.0
Hours per week             60.0
ID                   1344995623
Target                    >50K.
Name: 48842, dtype: object

In [51]:
#opcion con pandas 
p.index=p['ID'].values
w.index=w['ID'].values
union=w.join(p,how='outer', lsuffix='_w', rsuffix='_p')
union.iloc[np.where(union.ID_w!=union.ID_p)[0][0]]

In [65]:
agrupado=union.groupby('Target')

In [66]:
agrupado.Target.count()

Target
 <=50K     24720
 <=50K.    12435
 >50K       7841
 >50K.      3846
Name: Target, dtype: int64

In [70]:
union.dtypes

Workclass          object
fnlwgt            float64
Capital Gain      float64
Capital Loss      float64
Hours per week    float64
ID_w                int64
Target             object
Age               float64
Sex                object
ID_p              float64
Race               object
Country            object
Education          object
Education-Num     float64
Marital Status     object
Relationship       object
dtype: object

In [75]:
union['Target']=union.Target.str.replace('.','').values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [77]:
agrupado=union.groupby('Target')
agrupado.Target.count()

Target
 <=50K    37155
 >50K     11687
Name: Target, dtype: int64

In [83]:
#opcion usando apply 
def target_corregido(a):
  b=a.replace('.','')
  return(b)
df2=pd.DataFrame(union.Target).apply(target_corregido)
union['Target']=df2.values


In [94]:
#opcion 1
###np.isin(union.Target.values,[' >50K'])
#opcion 2
###union.Target==' >50K'
a=union.iloc[np.where(union.Target==' >50K')]

In [98]:
proporcion=len(a)/len(union)
print('proporcion = ' + str(np.round(proporcion*100)) + '%' )

proporcion = 24.0%


In [101]:
a.groupby('Sex').Target.count()

Sex
 Female    1769
 Male      9917
Name: Target, dtype: int64

In [110]:
b=pd.DataFrame(a.groupby('Sex').Target.count())
b

,Target
Sex,
Female,1769
Male,9917


In [115]:
b.iloc[0].values[0]/len(union)

0.0362180865221219

In [123]:
b.loc[' Female'].values[0]/len(union)

0.0362180865221219

In [124]:
b.loc[' Male'].values[0]/len(union)

0.20303830641033516